# HW2

### files for your reference

below link is our repository for the several part of the Assignment.
Since some steps will use this repository, run this one at first.

In [ ]:
## copy our 550Repository from github
!cd /content && git clone 'https://github.com/shlee55/550Repository.git'

Cloning into '550Repository'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2612 (delta 2), reused 0 (delta 0), pack-reused 2604
Receiving objects: 100% (2612/2612), 111.63 MiB | 59.82 MiB/s, done.
Resolving deltas: 100% (57/57), done.


### 1.  
### Executing TTR  
### adding XHTML & Content(with Sender and Title) columns into featureTable.tsv  
### creating part of Grover training input  

files needed  (you should put those files into the same folder with this notebook)
 - fradulent_emails.txt_meta.json  
 - featureTable.tsv  

files created
 - Grover_Fradulent.json (use in step 9)  
 - featureTable_temp.tsv (use in step 2 and step 10)


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=1c2ec3e5a19348e49f08b99e99fef4dbb363004751ddbab22b7d98f98157c1ab
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import tika
import json
from google.colab import files
import pandas as pd

In [ ]:
## extract content from XHTML using text tag ratio
def TTRtext(XHTML): 
    XHTML_line = XHTML.replace("\r\n","\n").replace("\r","\n").split('\n')
    content = ""
    for line in XHTML_line:
        TagsCount = 0
        while line.find("<") >= 0 or line.find(">") >= 0 : #true if any of the "<" or ">" is in the line
            ini = line.find("<")
            end = line.find(">")
            if ini == -1: # when the line doesn't include "<". keep strings after ">" sign as a text.
                line = line[end+1:]
            elif end == -1: # when the line doesn't include ">". keep strings before "<" sign as a text.
                line = line[:ini]
            elif ini < end: # regular shape 
                line = line[:ini]+line[end+1:] #remove the tag part from the line.
            else : # end > ini. it happens when we have some line like "...hello> yes right <next line..."
                line = line[end+1:]
            TagsCount += 1 # count tags
        line = line.replace("\xa0","").replace("\t","")
        TextCount = len(line.replace(" ", "")) # count text only (space is not counted)
        if TagsCount == 0:
            TR = TextCount
        else:
            TR = TextCount/TagsCount
        if TR >= 1:
            content += " " + line
    return(content)

## find the Sender and the Title from XHTML
def SenderAndTitle(XHTML): # get Sender and Title from HTML form
    XHTML_line = XHTML.replace("\r\n","\n").replace("\r","\n").split('\n')
    content = ""
    title = ""
    sender = ""
    for line in XHTML_line:
        if title == "":
            if line.find('<title>') >= 0:
                ini = line.find('<title>') + len('<title>')
                end = line.find('</title>')
                title = line[ini:end]
            elif line.find('<meta name="subject" content="') >= 0:
                ini = line.find('<meta name="subject" content="') + len('<meta name="subject" content="')
                end = line.find('" />')
                title = line[ini:end]
            elif line.find('<meta name="dc:title" content="') >= 0:
                ini = line.find('<meta name="dc:title" content="') + len('<meta name="dc:title" content="')
                end = line.find('" />')
                title = line[ini:end]

        if sender == "":
            if line.find('<meta name="Message:From-Name" content="') >= 0:
                ini = line.find('<meta name="Message:From-Name" content="') + len('<meta name="Message:From-Name" content="')
                end = line.find('" />')
                sender = line[ini:end]
            elif line.find('<meta name="Author" content="') >= 0:
                ini = line.find('<meta name="Author" content="') + len('<meta name="Author" content="')
                end = line.find('" />')
                sender = line[ini:end]
    return("From: " + sender + "\n" + "Title: " + title + "\n")

## add sender and title to the list of content
def AddSenderAndTitle(XHTMLList,contentList):
    FinalList = []
    length = len(XHTMLList)
    for i in range(length):
        FinalContent = SenderAndTitle(XHTMLList[i]) + contentList[i]
        FinalList.append(FinalContent)
    return(FinalList)

## generate list of XHTML and content from our parsed json file
## also generate part of the Grover input (fradulent email part of Grover)
def extractContentList(filename,SaveAsJSON): 
    XHTMLList = []
    contentList = []
    contentListDict = [] # list of Dictionary format is used for Grover input. 
    with open(filename,"r",encoding="utf8") as f:
        json_obj_list = json.load(f) # Read the file and convert it to a dictionary
    length = len(json_obj_list) # have to use len instead of key function because this is a list
    for i in range(length):
        json_obj = json_obj_list[i]
        try:
            XHTML = json_obj["X-TIKA:content"]
            EmailText = TTRtext(XHTML)
        except:
            EmailText = ""

        ## Grover input part
        if EmailText != "":
            XHTMLList.append(XHTML)
            contentList.append(EmailText)
            contentDict = {}
            contentDict["article"] = EmailText
            contentDict["label"] = "human" ## human or machine
            contentDict["split"] = "train"
            contentListDict.append(contentDict)

    with open(SaveAsJSON, 'w') as out_json_file:
        json.dump(contentListDict, out_json_file)
    files.download(SaveAsJSON)
    ContentWithHeaderList = AddSenderAndTitle(XHTMLList, contentList)
    return(XHTMLList, ContentWithHeaderList)

## add XHTML and EmailText columns on our TSV file.
def TSVwithXHTMLandContents(TSVfile, SaveAsTSV, XHTMLList, contentList):
    featureTable = pd.read_csv(TSVfile,sep="\t",index_col=[0])
    for i in range(len(featureTable)):
        EmailName = str(i) + ".json"
        featureTable.loc[featureTable.filename == EmailName,"XHTML"] = XHTMLList[i]
        featureTable.loc[featureTable.filename == EmailName,"EmailText"] = contentList[i] ## or contentList
    featureTable.to_csv(SaveAsTSV, sep = '\t')
    files.download(SaveAsTSV)
    return(featureTable)

In [ ]:
XHTMLList, contentList = extractContentList('fradulent_emails.txt_meta.json', ## input file for TTR
                                            'Grover_Fradulent.json') ## output file for Grover

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
featureTable = TSVwithXHTMLandContents("featureTable.tsv", ## input file
                                       "featureTable_temp.tsv", ## output file
                                       XHTMLList, 
                                       contentList)
featureTable.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,filename,AttackType:Reconnaissance,AttackType:SocialEngineering,AttackType:Malware,AttackType:CredentialPhishing,Attacker Title,Urgency of the attack email,DateAndTime:Date,DateAndTime:time,Attacker Offering,AttackerLocation:Country,AttackerLocation:Region,AttackerLocation:City,AttackerLocation:Country From Content,Attacker Relationship,AttackerSentiment:polarity,AttackerSentiment:subjectivity,AttackerLanguageStyle:misspelling,AttackerLanguageStyle:randomCapital,Estimated Age,Attacker IP Known As Phisher,Total Deaths and Injuries,Total Damage,Disaster Count,GDP per capita,Internet Use,Age dependency,URL Label,URL Length,URL Classification,XHTML,EmailText
0,0.json,False,True,False,True,soldier,2,2002-10-31,02:38,Money,Zimbabwe,Unknown,Unknown,Zimbabwe,stranger,0.01,0.51,2,3,31.0,unknown,7811.0,1655.428,70.0,1331.01,1.10,81.03,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: MR. JAMES NGOLA.\nTitle: URGENT BUSINESS...
1,1.json,False,True,True,False,no title exist,2,2002-10-31,11:10,Money,Nigeria,Unknown,Unknown,Nigeria,stranger,0.16,0.47,2,1,33.0,unknown,7811.0,1655.428,70.0,1598.82,0.32,86.80,NaN,short,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: Mr. Ben Suleman\nTitle: URGENT ASSISTANC...
2,10.json,True,True,False,False,no title exist,2,2001-11-06,16:52,Money,Unknown,Unknown,Unknown,Unknown,stranger,0.05,0.39,1,2,34.0,unknown,209137.0,7413.500,30.0,NaN,NaN,NaN,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: MR TEMI JOHNSON\nTitle: IMPORTANT\n IMPO...
3,100.json,True,False,False,False,no title exist,2,2003-02-06,15:43,Money,Switzerland,Valais,Sierre,Unknown,stranger,0.10,0.42,1,1,35.0,low,96784.0,11282.300,82.0,67385.30,65.10,47.81,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: stevens bangura\nTitle: confidential.\n ...
4,1000.json,False,False,True,True,prince,2,2004-07-11,10:50,Money,Brunei,Unknown,Unknown,Brunei,unknown,0.07,0.36,3,1,30.0,unknown,464473.0,48757.000,89.0,NaN,NaN,NaN,NaN,short,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: princefwbolkiah2000\nTitle: SEEKING YOUR...


### 2.
### generating input for GPT-2-simple (first 800 emails)

files needed  (you should put those files into the same folder with this notebook)
 - featureTable_temp.tsv

files created
 - Reconn_GPT2_InitialEmail_input.txt(use in step 3)  
 - SocEng_GPT2_InitialEmail_input.txt(use in step 3)  
 - Malware_GPT2_InitialEmail_input.txt(use in step 3)  
 - CrePhi_GPT2_InitialEmail_input.txt(use in step 3)


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import json
from google.colab import files
import pandas as pd

In [ ]:
# if the value of the URL Label column is "bad", then the attack type is malware.
#  otherwise, the priority is Credential Phishing, Malware, Social Engineering, and Reconnaissance
def InputForGPT2SimpleInitialEmail(featureTableTSV,SocEngInputTxt,MalwareInputTxt,ReconnInputTxt,CrePhiInputTxt):
    featureTable = pd.read_csv(featureTableTSV,sep="\t",index_col=[0])
    Reconn=[]
    SocEng=[]
    Malware=[]
    CrePhi=[]
    NoType=[]

    for i in range(len(featureTable)):
        if featureTable.loc[i,"URL Label"] == "bad":
            Malware.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "Malware"
        elif featureTable.loc[i,"AttackType:CredentialPhishing"] == True:
            CrePhi.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "CrePhi"
        elif featureTable.loc[i,"AttackType:Malware"] == True:
            Malware.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "Malware"
        elif featureTable.loc[i,"AttackType:SocialEngineering"] == True:
            SocEng.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "SocEng"
        elif featureTable.loc[i,"AttackType:Reconnaissance"] == True:
            Reconn.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "Reconn"
        else:
            NoType.append(featureTable.loc[i,"EmailText"]+"\n <|endoftext|>")
            featureTable.loc[i,"PickOneAttackType"] = "NoType"
    print("Reconn:", len(Reconn))
    print("SocEng:", len(SocEng))
    print("Malware:", len(Malware))
    print("CrePhi:", len(CrePhi))

    with open(SocEngInputTxt, 'w', encoding='utf8') as f:
        for item in SocEng:
            f.write("%s\n" % item)
        files.download(SocEngInputTxt)

    with open(MalwareInputTxt, 'w', encoding='utf8') as f:
        for item in Malware:
            f.write("%s\n" % item)
        files.download(MalwareInputTxt)

    with open(ReconnInputTxt, 'w', encoding='utf8') as f:
        for item in Reconn:
            f.write("%s\n" % item)
        files.download(ReconnInputTxt)

    with open(CrePhiInputTxt, 'w', encoding='utf8') as f:
        for item in CrePhi:
            f.write("%s\n" % item)
        files.download(CrePhiInputTxt)


In [ ]:
InputForGPT2SimpleInitialEmail("featureTable_temp.tsv",
                               "SocEng_GPT2_InitialEmail_input.txt",
                               "Malware_GPT2_InitialEmail_input.txt",
                               "Reconn_GPT2_InitialEmail_input.txt",
                               "CrePhi_GPT2_InitialEmail_input.txt")

Reconn: 775
SocEng: 740
Malware: 1085
CrePhi: 998


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3.
### (GPT2)Generating initial 800 emails
##### * GPT2 generates different result every time. So the result from the past execution results are stored in **"550Repository/GPT2Result/1. initial email"** folder, which is created previously.
------------------------------
#### Train a GPT-2 Text-Generating Model w/ GPU For Free
by Max Woolf  
Last updated: February 14th, 2021

------------------------------

  **to use trained model, go to this link and download below files,**
    **https://drive.google.com/drive/folders/1r0i9gUOaYoVOHQOxJchXhEYZC9Fb-SZv?usp=sharing**  
* checkpoint_SocEng355M.tar
* checkpoint_Malware355M.tar
* checkpoint_Reconn355M.tar
* checkpoint_CrePhi355M.tar

------------------------------  
  
  
files needed  (you should put those files into the same folder with this notebook)
 - SocEng_GPT2_InitialEmail_input.txt
 - Malware_GPT2_InitialEmail_input.txt
 - Reconn_GPT2_InitialEmail_input.txt
 - CrePhi_GPT2_InitialEmail_input.txt

files created
 - gpt2_gentext_YYYYMMDD_HHMMSS_SocEng355M.txt(use in step 4 and step9)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_Malware355M.txt(use in step 4 and step9)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_Reconn355M.txt(use in step 4 and step9)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_CrePhi355M.txt(use in step 4 and step9)  


In [ ]:
## restart runtime after you run this code
!pip install tensorflow==1.13.2

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

In [ ]:
## mount google drive to save your trained checkpoint or to use the trained model(checkpoint) in your google drive
gpt2.mount_gdrive() 

Mounted at /content/drive


In [ ]:
## generate GPT2 emails.
## if finetune == True, then it will run from finetune and generate checkpoint file, otherwise it will use trained checkpoint

def GenerateEmailsFromFinetune(AttackType, ModelSize, InputFile, finetune = True):
    RunName = AttackType + ModelSize

    ## Generating or loading the finetuned model
    if finetune == True: #running finetune to train model

        file_name = InputFile
        sess = gpt2.start_tf_sess()
        gpt2.finetune(sess,
                      dataset=file_name,
                      model_name=ModelSize,
                      steps=1000
                      restore_from='fresh',
                      run_name=RunName,
                      print_every=10
                      sample_every=200
                      save_every=500
                      )
        gpt2.copy_checkpoint_to_gdrive(run_name=RunName) 
    else: ## use trained model checkpoint to generate initial emails
        gpt2.copy_checkpoint_from_gdrive(run_name=RunName)
        sess = gpt2.start_tf_sess()
        gpt2.load_gpt2(sess, run_name=RunName)
    
    ## Generate emails
    gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}_'.format(datetime.utcnow()) + RunName + '.txt'
    gpt2.generate_to_file(sess,
                          destination_path=gen_file,
                          temperature=0.8,
                          nsamples=600
                          batch_size=10,
                          prefix = "From:", ## using prefix to keep our email form starting with "From:"
                          truncate = "<|endoftext|>",
                          run_name = RunName)
    files.download(gen_file)
    sess = gpt2.reset_session(sess)
    return(gen_file)


In [ ]:
ModelSize = "355M"

In [ ]:
# no need to run if using trained model
gpt2.download_gpt2(model_name=ModelSize)

In [ ]:
gen_file_SocEng = GenerateEmailsFromFinetune(AttackType = "SocEng", ## SocEng, Malware, Reconn, CrePhi
                                             ModelSize = ModelSize, ## 124M, 355M
                                             InputFile = "SocEng_GPT2_InitialEmail_input.txt", ## SocEng_GPT2_InitialEmail_input.txt, Malware_GPT2_InitialEmail_input.txt, Reconn_GPT2_InitialEmail_input.txt, CrePhi_GPT2_InitialEmail_input.txt
                                             finetune = True)

In [ ]:
gen_file_Malware = GenerateEmailsFromFinetune(AttackType = "Malware", ## SocEng, Malware, Reconn, CrePhi
                                             ModelSize = ModelSize, ## 124M, 355M
                                             InputFile = "Malware_GPT2_InitialEmail_input.txt", ## SocEng_GPT2_InitialEmail_input.txt, Malware_GPT2_InitialEmail_input.txt, Reconn_GPT2_InitialEmail_input.txt, CrePhi_GPT2_InitialEmail_input.txt
                                             finetune = True)

In [ ]:
gen_file_Reconn = GenerateEmailsFromFinetune(AttackType = "Reconn", ## SocEng, Malware, Reconn, CrePhi
                                             ModelSize = ModelSize, ## 124M, 355M
                                             InputFile = "Reconn_GPT2_InitialEmail_input.txt", ## SocEng_GPT2_InitialEmail_input.txt, Malware_GPT2_InitialEmail_input.txt, Reconn_GPT2_InitialEmail_input.txt, CrePhi_GPT2_InitialEmail_input.txt
                                             finetune = True)

In [ ]:
gen_file_CrePhi = GenerateEmailsFromFinetune(AttackType = "CrePhi", ## SocEng, Malware, Reconn, CrePhi
                                             ModelSize = ModelSize, ## 124M, 355M
                                             InputFile = "CrePhi_GPT2_InitialEmail_input.txt", ## SocEng_GPT2_InitialEmail_input.txt, Malware_GPT2_InitialEmail_input.txt, Reconn_GPT2_InitialEmail_input.txt, CrePhi_GPT2_InitialEmail_input.txt
                                             finetune = True)

### 4.
### remove unusable emails from initial emails

files needed  (you should put those files into the same folder with this notebook)
 - gpt2_gentext_YYYYMMDD_HHMMSS_SocEng355M.txt(here we use gpt2_gentext_20210406_212525_SocEng355M.txt)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_Malware355M.txt(here we use gpt2_gentext_20210406_193229_Malware355M.txt)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_Reconn355M.txt(here we use gpt2_gentext_20210406_205217_Reconn355M.txt)  
 - gpt2_gentext_YYYYMMDD_HHMMSS_CrePhi355M.txt(here we use gpt2_gentext_20210406_234706_CrePhi355M.txt)  

files created
 - SocEng_adj.csv, SocEng_adj.json(use in step 6)  
 - Malware_adj.csv, Malware_adj.json(use in step 6)  
 - Reconn_adj.csv, Reconn_adj.json(use in step 6)  
 - CrePhi_adj.csv, CrePhi_adj.json(use in step 6)  

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import pandas as pd
import json
from google.colab import files

In [ ]:
## from the previous step, we generated 600 emails per each attack type, even though we only need 200 emails per each attack type.
## using this method, remove some useless emails
def RemoveErrorEmail(txtFile):
    print("total number of emails:", len(txtFile))
    adjustedTxt = []
    a = 0
    b = 0
    c = 0 
    d = 0
    e = 0
    f = 0
    for content in txtFile:
        content = content.strip()
        if content.count("\n") > 10: 
            a += 1
            continue
        elif content.find("Title:") < 0:
            b += 1
            continue
        elif len(content.split()) < 20:
            c += 1
            continue
        elif len(content.split()) > 500:
            d += 1
            continue
        elif content.count('@') > 10:
            e += 1
            continue
        else:
            sender = content[5:content.find("\n")].strip()
            contentSL = content[(content.find("\n")+2):].strip() # content from second line
            title = contentSL[6:contentSL.find("\n")].strip()
            if len(sender) < 2:
                f += 1
                continue
            elif len(title) < 2:
                f += 1
                continue
            elif len(title) > 50:
                f += 1
                continue
            adjustedTxt.append("<originalEmail>\n" + content + "\n\n<replyEmail>\nTitle: Re: " + title + "\nDear " + sender + "\n")
    print("a:",a,", b:", b,", c:", c,", d:", d,", e:", e,", f:", f)
    print("total number of emails:", len(adjustedTxt))
    return(adjustedTxt)

## select GPT2 generated txt file to run above method(RemoveErrorEmail() and create result csv and json file.
def generateCSV(file,SaveAsCSV,SaveAsJSON):
    with open(file,"r",encoding='utf8') as f:
        contents = f.read().split("\n====================\n")
    print(len(contents))
    contents_adj = RemoveErrorEmail(contents)
    df = pd.DataFrame(contents_adj)
    df.to_csv(SaveAsCSV, index=False)

    ## input for interactive GPT2 (reply email)
    MachineTrainingList = []
    for i in range(len(contents_adj)):
        MTDict = {}
        MTDict["context"] = contents_adj[i].strip()
        MachineTrainingList.append(MTDict)          
    with open(SaveAsJSON, 'w') as out_json_file:
        json.dump(MachineTrainingList, out_json_file)
    files.download(SaveAsCSV)
    files.download(SaveAsJSON)


In [ ]:
ReconnInitial = generateCSV('gpt2_gentext_20210406_205217_Reconn355M.txt','Reconn_adj.csv','Reconn_adj.json')

601
total number of emails: 601
a: 0 , b: 47 , c: 73 , d: 19 , e: 0 , f: 148
total number of emails: 314


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
MalwareInitial = generateCSV('gpt2_gentext_20210406_193229_Malware355M.txt','Malware_adj.csv','Malware_adj.json')

601
total number of emails: 601
a: 0 , b: 111 , c: 3 , d: 106 , e: 3 , f: 69
total number of emails: 309


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
CrePhiInitial = generateCSV('gpt2_gentext_20210406_234706_CrePhi355M.txt','CrePhi_adj.csv','CrePhi_adj.json')

801
total number of emails: 801
a: 0 , b: 309 , c: 26 , d: 164 , e: 0 , f: 63
total number of emails: 239


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
SocEngInitial = generateCSV("gpt2_gentext_20210406_212525_SocEng355M.txt",'SocEng_adj.csv','SocEng_adj.json')

601
total number of emails: 601
a: 0 , b: 25 , c: 3 , d: 145 , e: 0 , f: 99
total number of emails: 329


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 5.
### generating reply email corpus using Enron Email Corpus
#### * download emails.csv file before running this step!
##### (you should download this file from below link, since it is more than 1.3GB)  

-----
https://www.kaggle.com/wcukierski/enron-email-dataset  
you can downloade the Enron dataset here!!  

-----
  
files needed  (you should put those files into the same folder with this notebook)
 - emails.csv 

files created
 - SocEngEnronEmails.txt(use in step 6 and step 7)  
 - MalwareEnronEmails.txt(use in step 6 and step 7)  
 - ReconnEnronEmails.txt(use in step 6 and step 7)  
 - CrePhiEnronEmails.txt(use in step 6 and step 7)  

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
!pip install tika

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
!python -m spacy download en_core_web_sm ## restart the colab runtime and run next code

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import tika
import json
import os
from tika import unpack, parser
import pandas as pd
from google.colab import files
import spacy
from datetime import datetime
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm") ## or "en_core_web_sm"

In [ ]:
## extract reply email from Enron Email Corpus
def extractRe(string): # extract reply email only
    content = ""
    if string.lower().find("subject: re: re:") >= 0:
        pass
    elif string.lower().find("subject: re: fw:") >= 0:
        pass
    elif string.lower().find("subject: re: fwd:") >= 0:
        pass
    elif string.lower().find("subject: re:") >= 0:
        email = parser.from_buffer(string)
        try:
            content = email["content"].strip()
        except:
            pass
    return(content)

## determine whether the email content is about social engineering or not
## modified some code from HW1
def get_socialEngineering(email): 
    TorF = False
# define a list of keywords
    keysets = {"urgent need","my children", "urgency","urgent","children", "child", "son", "duaghter", "asap", "as soon as possible"}
    try:
        for word in email.split():
            if word.lower() in keysets:
                TorF = True
                break
    except KeyError:
        pass
    return(TorF)

## determine whether the email content is about Malware or not
## modified some code from HW1
def get_Malware(email): ## modified some code from HW1
    TorF = False
    matcher = Matcher(nlp.vocab)
    url = [
        [{"LIKE_URL":True}] ## bank account, banking account, bank accounts, banking accounts
    ]
    matcher.add("Malware",url)
    doc = nlp(email)
    matches = matcher(doc)
    if matches: # if not empty
        TorF = True
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
    return(TorF)

## determine whether the email content is about Credential Phishing or not
## modified some code from HW1
def get_Credential_Phishing(email): 
    """detect key words in content and 
    determines whether it is a credential phishing"""
    TorF = False
    matcher = Matcher(nlp.vocab)
    credPhish = [
        [{"LOWER":"date"},{"LOWER":"of"},{"LOWER":"birth"}], ## search for date of birth
        [{"LOWER":"place"},{"LOWER":"of"},{"LOWER":"birth"}], ## place of birth? I am not sure if we need it or not
        [{"LOWER":"social"},{"LOWER":"security"}], ## social security
        [{"LEMMA":"account"},{"LOWER":"number"}], ## account number
        [{"LEMMA":"phone"},{"LOWER":"number"}], ## phone number
        [{"LEMMA":"bank"},{"LEMMA":"account"}] ## bank account, banking account, bank accounts, banking accounts
    ]
    matcher.add("Cred_Phishing",credPhish)
    doc = nlp(email)
    matches = matcher(doc)
    if matches: ## if not empty => it is credential phishing
        TorF = True
    return(TorF)

## determine whether the email content is about Reconnaissance or not
## modified some code from HW1
def get_reconnaissance(email):
    TorF = False
    matcher = Matcher(nlp.vocab)
    reconn = [
        [{"LOWER":"i"},{"LOWER":"am"},{"LOWER":"not"},{"LOWER":"interested"}], ## I am not interested
        [{"LOWER":"in"},{"LOWER":"response"},{"LOWER":"to"},{"LOWER":"your"}], ## In response to your
        [{"LOWER":"yes"},{"LOWER":"i'm"},{"LOWER":"interested"}], ## Yes, i'm interested
        [{"LOWER":"i"},{"LOWER":"am"},{"LOWER":"not"},{"LOWER":"sure"}], ## I am not sure
        [{"LOWER":"give"},{"LOWER":"me"},{"LOWER":"some"},{"LOWER":"time"}], ## Give me some time
        [{"LOWER":"yes"},{"LOWER":"i"},{"LOWER":"am"},{"LOWER":"interested"}], ## yes I am interested
        [{"LOWER":"thank"},{"LOWER":"you"},{"LOWER":"for"},{"LOWER":"your"},{"LOWER":"email"}] ## thank you for your email
    ]
    matcher.add("Reconn",reconn)
    doc = nlp(email)
    matches = matcher(doc)
    if matches:
        TorF = True
    return(TorF)


## parse Reply emails using tika
def ParseReplyEmail(emails): 
    ReplyEmailsParsed = []
    for i in range(len(emails)):
        email = extractRe(emails.loc[i,"message"])
        ReplyEmailsParsed.append(email)
        if i % 500 == 0:
            print(i)
    print(len(ReplyEmailsParsed))
    return(ReplyEmailsParsed)

## remove some useless email contents
def KeepUsefulEmails(EmailList):
    UsefulEmails = []
    for i in range(len(EmailList)):
        email = EmailList[i]
        email = email.replace("\n\n\n\n","\n\n").replace("\n\n\n","\n\n").replace("\n\n","<MultiNewLine>").replace("\n"," ").replace("<MultiNewLine>","\n\n")
        if email.count("\t") < 10 and email.count("\n\n") < 10 and email.count("@") < 5 and len(email) > 100:
            UsefulEmails.append(email + "\n\n\n <|endoftext|> \n\n\n")
        if i % 500 == 0:
            print(i)
    print(len(UsefulEmails))
    
    with open('ReplyEmails.txt', 'w', encoding='utf8') as f:
        for item in UsefulEmails:
            f.write("%s\n" % item)

    return(UsefulEmails)

## distribute attacker type using above methods
def AttackerTypeSorting(Emails,SaveAsTxtSocEng,SaveAsTxtMalware,SaveAsTxtReconn,SaveAsTxtCrePhi):
    SocEngList = []
    MalwareList = []
    ReconnList = []
    CrePhiList = []
    
    for i in range(len(Emails)):
        if get_socialEngineering(Emails[i]) == True:
            SocEngList.append(Emails[i])
        if get_Malware(Emails[i]) == True:
            MalwareList.append(Emails[i])
        if get_reconnaissance(Emails[i]) == True:
            ReconnList.append(Emails[i])
        if get_Credential_Phishing(Emails[i]) == True:
            CrePhiList.append(Emails[i])
        if i % 500 == 0:
            print(i)    
    print("SocEng:", len(SocEngList))
    print("Malware:", len(MalwareList))
    print("Reconn:", len(ReconnList))
    print("CrePhi:", len(CrePhiList))

    with open(SaveAsTxtSocEng, 'w', encoding='utf8') as f:
        for item in SocEngList:
            f.write("%s\n" % item)
        files.download(SaveAsTxtSocEng)

    with open(SaveAsTxtMalware, 'w', encoding='utf8') as f:
        for item in MalwareList:
            f.write("%s\n" % item)
        files.download(SaveAsTxtMalware)

    with open(SaveAsTxtReconn, 'w', encoding='utf8') as f:
        for item in ReconnList:
            f.write("%s\n" % item)
        files.download(SaveAsTxtReconn)

    with open(SaveAsTxtCrePhi, 'w', encoding='utf8') as f:
        for item in CrePhiList:
            f.write("%s\n" % item)
        files.download(SaveAsTxtCrePhi)


In [ ]:
emails = pd.read_csv("emails.csv")

In [ ]:
parsed = ParseReplyEmail(emails)

2021-04-08 19:03:34,008 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


0


2021-04-08 19:03:35,132 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-04-08 19:03:35,618 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [ ]:
ReplyEmail = KeepUsefulEmails(parsed)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

In [ ]:
AttackerTypeSorting(ReplyEmail,
                    "SocEngEnronEmails.txt",
                    "MalwareEnronEmails.txt",
                    "ReconnEnronEmails.txt",
                    "CrePhiEnronEmails.txt")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
SocEng: 634
Malware: 4041
Reconn: 422
CrePhi: 629


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 6.
### (GPT2)Generating Interactive Email (victim -> attacker)
##### * GPT2 generates different result every time. So the result from the past execution results are stored in **"550Repository/GPT2Result/2. reply email"** folder, which is created previously. You can move those file into your current folder and run next step.

------------------------------

by Ben Gumser's github
https://github.com/bengum/gpt-2.git 

------------------------------

  **to use trained model, go to this link and download below files,**
  **https://drive.google.com/drive/folders/1r0i9gUOaYoVOHQOxJchXhEYZC9Fb-SZv?usp=sharing**  
* checkpoint_SocEngEnron355M.tar
* heckpoint_MalwareEnron355M.tar
* checkpoint_ReconnEnron355M.tar
* checkpoint_CrePhiEnron355M.tar

------------------------------  

files needed  (you should put those files into the same folder with this notebook)
- SocEngEnronEmails.txt, SocEng_adj.json  
- MalwareEnronEmails.txt, Malware_adj.json  
- ReconnEnronEmails.txt, Reconn_adj.json
- CrePhiEnronEmails.txt, CrePhi_adj.json  

files created
 - SocEng_adj_output.json(use in step 7 and step 8)  
 - Malware_adj_output.json(use in step 7 and step 8)  
 - Reconn_adj_output.json(use in step 7 and step 8)  
 - CrePhi_adj_output.json(use in step 7 and step 8)  

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
## restart runtime after you run this code
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7MB 32kB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.2MB 49.6MB/s 
     |████████████████████████████████| 368kB 43.8MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# Clones my Github into this Colab notebook
!cd /content && rm -rf gpt2 && git clone 'https://github.com/bengum/gpt-2.git'
%cd '/content/gpt-2'
!pip install -r requirements.txt
%cd '/content/gpt-2/src'
from interactive_samples_from_json import interact_model_from_json
%cd '/content'

fatal: destination path 'gpt-2' already exists and is not an empty directory.
/content/gpt-2
/content/gpt-2/src
/content


In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
## generate GPT2 emails.
## if finetune == True, then it will run from finetune and generate checkpoint file, otherwise it will use trained checkpoint
def GenerateReplyEmailsModelFromFinetune(AttackType, ModelSize, InputFile, ContextFile, OutputFile, finetune = True):
    RunName = AttackType + ModelSize
    
    ## Generating or loading the finetuned model
    if finetune == True: #running finetune to train model
        file_name = InputFile
        sess = gpt2.start_tf_sess()
        gpt2.finetune(sess,
                      dataset=file_name,
                      model_name=ModelSize,
                      steps=1000,
                      restore_from='fresh',
                      run_name=RunName,
                      print_every=10,
                      sample_every=200,
                      save_every=500
                      )
        gpt2.copy_checkpoint_to_gdrive(run_name=RunName) 
    else: ## use trained model checkpoint to generate initial emails
        gpt2.copy_checkpoint_from_gdrive(run_name=RunName)
        sess = gpt2.start_tf_sess()
        gpt2.load_gpt2(sess, run_name=RunName)
        
    ## Generate emails
    ## Keep the batch size at 1 unless you want to use the same context for the whole batch
    interact_model_from_json(model_name=RunName, 
                             nsamples=200, 
                             batch_size=1,
                             top_k=40, 
                             length=None, 
                             temperature=1.0, 
                             models_dir='checkpoint', 
                             context_file = ContextFile, 
                             prefix='<|startoftext|>', 
                             include_prefix=True, 
                             truncate = '<|endoftext|>', 
                             output_file = OutputFile
                             )
    files.download(OutputFile)
    sess = gpt2.reset_session(sess)
    return(OutputFile)    

In [ ]:
ModelSize = "355M"

In [ ]:
# no need to run if using trained model
gpt2.download_gpt2(model_name=ModelSize)

Fetching checkpoint: 1.05Mit [00:00, 396Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.32Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 405Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [01:54, 12.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 224Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.57Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 7.53Mit/s]


In [ ]:
gen_file_SocEng_reply = GenerateReplyEmailsModelFromFinetune(AttackType = "SocEngEnron", ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "SocEngEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "SocEng_adj.json", ## SocEng_adj.json, Malware_adj.json, Reconn_adj.json, CrePhi_adj.json
                                     OutputFile = "SocEng_adj_output.json", ## SocEng_adj_output.json, Malware_adj_output.json, Reconn_adj_output.json, CrePhi_adj_output.json
                                     finetune = True)
                                     

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 your 
 need

I need  any reason
 

To this ?
 


 request
Dear ?

for this account
My
 action
My   to 
 This ?

   
My contact with this situation

 your immediate
 ?
 I 
  

 

======================================== SAMPLE 63 ========================================
<originalEmail>
From: bahram el Sisi
Title: URGENT
 URGENT Attn/Pls Dear Sir/Madam, I am Barrister Bahram el Sisi, a Solicitor. I was a Contractor with the Federal Government of Nigeria before my retirement in early 2001. One of my colleagues and I were involved in the contract award and recommendation review with a foreign contracting firm in the Federal Ministry of Rail Transport and Communications (FMRC) here in Nigeria for the supply of ammunations, hard/soft wares,phamaceuticals/medical items,light and heavy duty vehicles, ammunations procured from United Nations  ATOMI and other international agencies. We were involved in supplying equipment for the purchase of aircraft,phantom 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
gen_file_SocEng_reply = GenerateReplyEmailsModelFromFinetune(AttackType = "MalwareEnron", ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "MalwareEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "Malware_adj.json", ## SocEng_adj.json, Malware_adj.json, Reconn_adj.json, CrePhi_adj.json
                                     OutputFile = "Malware_adj_output.json", ## SocEng_adj_output.json, Malware_adj_output.json, Reconn_adj_output.json, CrePhi_adj_output.json
                                     finetune = True)

In [ ]:
gen_file_SocEng_reply = GenerateReplyEmailsModelFromFinetune(AttackType = "ReconnEnron", ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "ReconnEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "Reconn_adj.json", ## SocEng_adj.json, Malware_adj.json, Reconn_adj.json, CrePhi_adj.json
                                     OutputFile = "Reconn_adj_output.json", ## SocEng_adj_output.json, Malware_adj_output.json, Reconn_adj_output.json, CrePhi_adj_output.json
                                     finetune = True)

In [ ]:
gen_file_SocEng_reply = GenerateReplyEmailsModelFromFinetune(AttackType = "CrePhiEnron", ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "CrePhiEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "CrePhi_adj.json", ## SocEng_adj.json, Malware_adj.json, Reconn_adj.json, CrePhi_adj.json
                                     OutputFile = "CrePhi_adj_output.json", ## SocEng_adj_output.json, Malware_adj_output.json, Reconn_adj_output.json, CrePhi_adj_output.json
                                     finetune = True)

### 7.
### (GPT2)Generating Interactive Email (attacker -> victim)
##### * GPT2 generates different result every time. So the result from the past execution results are stored in **"550Repository/GPT2Result/3. reply reply email"** folder, which is created previously. You can move those file into your current folder and run next step.

------------------------------

by Ben Gumser's github 

------------------------------

  **to use trained model, go to this link and download below files,**
  **https://drive.google.com/drive/folders/1r0i9gUOaYoVOHQOxJchXhEYZC9Fb-SZv?usp=sharing**  
* checkpoint_SocEngEnron355M.tar
* heckpoint_MalwareEnron355M.tar
* checkpoint_ReconnEnron355M.tar
* checkpoint_CrePhiEnron355M.tar

------------------------------  

files needed  (you should put those files into the same folder with this notebook)
 - SocEngEnronEmails.txt, SocEng_adj_output.json   
 - ReconnEnronEmails.txt, Reconn_adj_output.json  
 - MalwareEnronEmails.txt, Malware_adj_output.json  
 - CrePhiEnronEmails.txt, CrePhi_adj_output.json  


files created
 - SocEng_Reply_adj.json, SocEng_Reply_adj_output.json(use in step 8)  
 - Malware_Reply_adj.json, Malware_Reply_adj_output.json(use in step 8)  
 - Reconn_Reply_adj.json, Reconn_Reply_adj_output.json(use in step 8)  
 - CrePhi_Reply_adj.json, CrePhi_Reply_adj_output.json(use in step 8)  

In [ ]:
### every step will be starting with import part in case of running from the specific step. 
## restart runtime after you run this code
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.7MB 45kB/s 
     |████████████████████████████████| 3.2MB 49.4MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 368kB 56.7MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import json

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# Clones my Github into this Colab notebook
!cd /content && rm -rf gpt2 && git clone 'https://github.com/bengum/gpt-2.git'
%cd '/content/gpt-2'
!pip install -r requirements.txt
%cd '/content/gpt-2/src'
from interactive_samples_from_json import interact_model_from_json
%cd '/content'

Cloning into 'gpt-2'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 283 (delta 33), reused 44 (delta 29), pack-reused 233
Receiving objects: 100% (283/283), 4.39 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/gpt-2
     |████████████████████████████████| 92kB 8.8MB/s 
     |████████████████████████████████| 604kB 36.4MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=41a9933320dc61df16c220ced584b58aae551aa9abfa6f1071702d5f364e68fc
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534404 sha256=1dd682ed14fd98f7e7c323e82

/content/gpt-2/src
/content


In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
## generate GPT2 emails.
## if finetune == True, then it will run from finetune and generate checkpoint file, otherwise it will use trained checkpoint
def GenerateReReEmailsModelFromFinetune(AttackType, ModelSize, InputFile, ContextFile, OutputFile, finetune = True):
    RunName = AttackType + ModelSize
    
    ## Generating or loading the finetuned model
    if finetune == True: #running finetune to train model
        file_name = InputFile
        sess = gpt2.start_tf_sess()
        gpt2.finetune(sess,
                      dataset=file_name,
                      model_name=ModelSize,
                      steps=1000,
                      restore_from='fresh',
                      run_name=RunName,
                      print_every=10,
                      sample_every=200,
                      save_every=500
                      )
        gpt2.copy_checkpoint_to_gdrive(run_name=RunName) 
    else: ## use trained model checkpoint to generate initial emails
        gpt2.copy_checkpoint_from_gdrive(run_name=RunName)
        sess = gpt2.start_tf_sess()
        gpt2.load_gpt2(sess, run_name=RunName)
    
    ## Generate emails
    ## Keep the batch size at 1 unless you want to use the same context for the whole batch
    interact_model_from_json(model_name=RunName, 
                             nsamples=200, 
                             batch_size=1,
                             top_k=40, 
                             length=None, 
                             temperature=1.0, 
                             models_dir='checkpoint', 
                             context_file = ContextFile, 
                             prefix='<|startoftext|>', 
                             include_prefix=True, 
                             truncate = '<|endoftext|>', 
                             output_file = OutputFile
                             )
    files.download(OutputFile)
    sess = gpt2.reset_session(sess)
    return(OutputFile)    

In [ ]:
## modifying input for third 800 emails GPT2 model.
def ReplyEmailContextforReRe(ReplyEmail,SaveAsJSON):
    ReplyEmailPD = pd.read_json (ReplyEmail)
    ContextFileList = []
    for i in range(len(ReplyEmailPD)):
        body = ReplyEmailPD.iloc[i,0]
        bodyA = body[body.find("\n")+1:]## remove "<originalEmail>"
        bodyB = bodyA[bodyA.find("\n")+1:]## remove Sender
        bodyC = bodyB[bodyB.find("\n")+1:]## remove Title
        bodyReply = body[body.find("<replyEmail>"):]
        Sender = bodyA[len("From: "):bodyA.find("\n")]
        title = bodyB[len("Title: "):bodyB.find("\n")]
        bodyReply = bodyReply.strip() + "\n\n<ReReEmail>\nFrom: " + Sender + "\nTitle: Re: Re: " + title + "\n"
        ContextDict = {}
        ContextDict["context"] = bodyReply
        ContextFileList.append(ContextDict)          
    with open(SaveAsJSON, 'w') as out_json_file:
        json.dump(ContextFileList, out_json_file)
    files.download(SaveAsJSON)
    return(ContextFileList)

In [ ]:
ModelSize = "355M"

In [ ]:
# no need to run if using trained model
gpt2.download_gpt2(model_name=ModelSize)

In [ ]:
ReplyEmailDict = ReplyEmailContextforReRe('SocEng_adj_output.json','SocEng_Reply_adj.json')
ReplyEmailDict = ReplyEmailContextforReRe('Malware_adj_output.json','Malware_Reply_adj.json')
ReplyEmailDict = ReplyEmailContextforReRe('Reconn_adj_output.json','Reconn_Reply_adj.json')
ReplyEmailDict = ReplyEmailContextforReRe('CrePhi_adj_output.json','CrePhi_Reply_adj.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
gen_file_SocEng_rere = GenerateReReEmailsModelFromFinetune(AttackType = "SocEngEnron",  ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "SocEngEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "SocEng_Reply_adj.json", ## SocEng_Reply_adj.json, Malware_Reply_adj.json, Reconn_Reply_adj.json, CrePhi_Reply_adj.json
                                     OutputFile = "SocEng_Reply_adj_output.json", ## SocEng_Reply_adj_output.json, Malware_Reply_adj_output.json, Reconn_Reply_adj_output.json, CrePhi_Reply_adj_output.json
                                     finetune = False)
                                     

In [ ]:
gen_file_SocEng_rere = GenerateReReEmailsModelFromFinetune(AttackType = "MalwareEnron",  ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "MalwareEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "Malware_Reply_adj.json", ## SocEng_Reply_adj.json, Malware_Reply_adj.json, Reconn_Reply_adj.json, CrePhi_Reply_adj.json
                                     OutputFile = "Malware_Reply_adj_output.json", ## SocEng_Reply_adj_output.json, Malware_Reply_adj_output.json, Reconn_Reply_adj_output.json, CrePhi_Reply_adj_output.json
                                     finetune = False)
                                     

In [ ]:
gen_file_SocEng_rere = GenerateReReEmailsModelFromFinetune(AttackType = "ReconnEnron",  ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "ReconnEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "Reconn_Reply_adj.json", ## SocEng_Reply_adj.json, Malware_Reply_adj.json, Reconn_Reply_adj.json, CrePhi_Reply_adj.json
                                     OutputFile = "Reconn_Reply_adj_output.json", ## SocEng_Reply_adj_output.json, Malware_Reply_adj_output.json, Reconn_Reply_adj_output.json, CrePhi_Reply_adj_output.json
                                     finetune = False)
                                     

In [ ]:
gen_file_SocEng_rere = GenerateReReEmailsModelFromFinetune(AttackType = "CrePhiEnron",  ## SocEngEnron, MalwareEnron, ReconnEnron, CrePhiEnron
                                     ModelSize = ModelSize, ## 124M, 355M
                                     InputFile = "CrePhiEnronEmails.txt", ## SocEngEnronEmails.txt, MalwareEnronEmails.txt, ReconnEnronEmails.txt, CrePhiEnronEmails.txt
                                     ContextFile = "CrePhi_Reply_adj.json", ## SocEng_Reply_adj.json, Malware_Reply_adj.json, Reconn_Reply_adj.json, CrePhi_Reply_adj.json
                                     OutputFile = "Reconn_Reply_adj_output.json", ## SocEng_Reply_adj_output.json, Malware_Reply_adj_output.json, Reconn_Reply_adj_output.json, CrePhi_Reply_adj_output.json
                                     finetune = False)
                                     

### 8.
### Add features (Attacker Face, Victim Face, and Phish Iris Image) into GPT2 generated emails

* Need to run **caption, FaceGenerator, Phish Iris Image** prior to this step and bring files into current folder.  
(**Caption:** TEAM_MIMECRAFT_DSCI550_HW_EXTRACT\new_attacks\caption\caption.py)  
(**FaceGenerator:** TEAM_MIMECRAFT_DSCI550_HW_EXTRACT\new_attacks\Face Generator\FaceGeneratorDCGAN.py)  
(**Phish Iris:** TEAM_MIMECRAFT_DSCI550_HW_EXTRACT\new_attacks\Phish Iris Images\FaceGeneratorDCGAN.py)
After running them, bring the final input(Social_Engineering-800_result.json, etc.) on this folder.
* Instead of running caption, use result file from our github repository (https://github.com/shlee55/550Repository.git).   
Path of the file: 550Repository/caption

files needed  (you should put those files into the same folder with this notebook)
 - SocEng_adj_output.json, SocEng_Reply_adj_output.json  
 - Malware_adj_output.json, Malware_Reply_adj_output.json  
 - Reconn_adj_output.json, Reconn_Reply_adj_output.json  
 - CrePhi_adj_output.json, CrePhi_Reply_adj_output.json  
 - Social_Engineering-800_result.json(github-result from Fake Phish Iris image and Caption)  
 - Malware-800_result.json(github-result from Fake Phish Iris image and Caption)  
 - Reconnaissance-800_result.json(github-result from Fake Phish Iris image and Caption)  
 - Credential_Phishing-800_result.json(github-result from Fake Phish Iris image and Caption)  


files created
 - SocEng_GPT2Emails.csv  
 - Malware_GPT2Emails.csv  
 - Reconn_GPT2Emails.csv  
 - CrePhi_GPT2Emails.csv  
 - GPT2Emails.tsv(use in step 9 and step 10)

In [ ]:
!cd /content && git clone 'https://github.com/shlee55/550Repository.git'

fatal: destination path '550Repository' already exists and is not an empty directory.


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
from google.colab import files
import json
import pandas as pd
import os

In [ ]:

## generating table for all 2400 emails from GPT2. 
## Table is consist of four columns(index, initial email, second email, and third email) and 800 rows (200 for each attack type)
def TotalEmailTable(AttackType, ReEmailJson,ReReEmailJson,SaveAsCSV):
    ReEmailResult = pd.read_json(ReEmailJson)
    ReReEmailResult = pd.read_json(ReReEmailJson)
    if len(ReEmailResult) != len(ReReEmailResult):
        print("result not matched, check the email result sets")
        return(None)
    else:
        TotalEmails = pd.DataFrame()
        IndexList = []
        FirstList = []
        SecondList = []
        ThirdList = []
        for i in range(len(ReEmailResult)):
            IndexNumber = ("000"+str(i+1))[-3:]
            ReEmail = ReEmailResult.iloc[i,0]
            ReReEmail = ReReEmailResult.iloc[i,0]
            first = ReEmail[ReEmail.find("<originalEmail>"):ReEmail.find("<replyEmail>")]
            second = ReReEmail[ReReEmail.find("<replyEmail>"):ReReEmail.find("<ReReEmail>")]
            third = ReReEmail[ReReEmail.find("<ReReEmail"):]
            IndexList.append(AttackType + ". " + IndexNumber)
            FirstList.append(first)
            SecondList.append(second)
            ThirdList.append(third)
        TotalEmails["index"] = IndexList
        TotalEmails["InitialEmail"] = FirstList
        TotalEmails["SecondEmail"] = SecondList
        TotalEmails["ThirdEmail"] = ThirdList
        TotalEmails.to_csv(SaveAsCSV, index = True)
        files.download(SaveAsCSV)
        return(TotalEmails)

## generating table for image path and its caption description
def ImageAndCaption(SocEng,Malware,Reconn,CrePhi):
#    path = "/content/550Repository/caption/"

    ICSocEng = pd.read_json(SocEng)
    ICSocEng = ICSocEng.sort_values(by=["img_path"]).reset_index(drop = True)

    ICMalware = pd.read_json(Malware)
    ICMalware = ICMalware.sort_values(by=["img_path"]).reset_index(drop = True)

    ICReconn = pd.read_json(Reconn)
    ICReconn = ICReconn.sort_values(by=["img_path"]).reset_index(drop = True)

    ICCrePhi = pd.read_json(CrePhi)
    ICCrePhi = ICCrePhi.sort_values(by=["img_path"]).reset_index(drop = True)

    return(ICSocEng[0:200],ICMalware[0:200],ICReconn[0:200],ICCrePhi[0:200])

## generating table for face image path
def FaceTable():
    path, dirs, files = next(os.walk("/content/550Repository/Face_generator-1600 result"))
    files.sort()
    length = len(files)
    AttackerFace = files[:800]
    for i in range(len(AttackerFace)):
        AttackerFace[i] = "Face_generator-1600 result/" + AttackerFace[i]
    VictimFace = files[800:1600]
    for i in range(len(VictimFace)):
        VictimFace[i] = "Face_generator-1600 result/" + VictimFace[i]

    AttackerFaceTable = pd.DataFrame({'col':AttackerFace})
    VictimFaceTable = pd.DataFrame({'col':VictimFace})
    return(AttackerFaceTable,VictimFaceTable)

## combine all those tables from above. 
## table consist of 5 columns ("AttackType_index","EmailText","SenderFace","PhishIrisPath","Caption") and 2400 rows
def TSVform(SocEng,Malware,Reconn,CrePhi,SocEngJson,MalwareJson,ReconnJson,CrePhiJson,SaveAsTSV):
    ICSocEng, ICMalware, ICReconn, ICCrePhi = ImageAndCaption(SocEngJson,MalwareJson,ReconnJson,CrePhiJson)
    AttackerFaceTable,VictimFaceTable = FaceTable()
    ConcatTable = pd.concat([SocEng,Malware,Reconn,CrePhi],ignore_index=True)
    ICtable = pd.concat([ICSocEng, ICMalware, ICReconn, ICCrePhi],ignore_index=True)

    InitialTable = ConcatTable.loc[:,["index","InitialEmail"]]
    InitialTable = pd.concat([InitialTable,AttackerFaceTable,ICtable], axis = 1)
    for i in range(len(InitialTable)):
        InitialTable.iloc[i,0] += "-1"
        InitialTable.iloc[i,1] = InitialTable.iloc[i,1][(InitialTable.iloc[i,1].find("\n")+1):]
    
    SecondTable = ConcatTable.loc[:,["index","SecondEmail"]]
    SecondTable = pd.concat([SecondTable,VictimFaceTable], axis = 1)
    for i in range(len(SecondTable)):
        SecondTable.iloc[i,0] += "-2"
        SecondTable.iloc[i,1] = SecondTable.iloc[i,1][(SecondTable.iloc[i,1].find("\n")+1):]

    ThirdTable = ConcatTable.loc[:,["index","ThirdEmail"]]
    ThirdTable = pd.concat([ThirdTable,AttackerFaceTable], axis = 1)
    for i in range(len(ThirdTable)):
        ThirdTable.iloc[i,0] += "-3"
        ThirdTable.iloc[i,1] = ThirdTable.iloc[i,1][(ThirdTable.iloc[i,1].find("\n")+1):]

    InitialTable.columns = ["AttackType_index","EmailText","SenderFace","PhishIrisPath","Caption"]
    SecondTable.columns = ["AttackType_index","EmailText","SenderFace"]
    ThirdTable.columns = ["AttackType_index","EmailText","SenderFace"]

    TSVFormTable = pd.concat([InitialTable,SecondTable,ThirdTable], axis=0, ignore_index=True)
    TSVFormTable.to_csv(SaveAsTSV, sep = '\t')
    files.download(SaveAsTSV)
    return(TSVFormTable)


In [ ]:
SocEngGPT2Total = TotalEmailTable("SocEng","SocEng_adj_output.json", "SocEng_Reply_adj_output.json","SocEng_GPT2Emails.csv")
MalwareGPT2Total = TotalEmailTable("Malware","Malware_adj_output.json", "Malware_Reply_adj_output.json","Malware_GPT2Emails.csv")
ReconnGPT2Total = TotalEmailTable("Reconn","Reconn_adj_output.json", "Reconn_Reply_adj_output.json","Reconn_GPT2Emails.csv")
CrePhiGPT2Total = TotalEmailTable("CrePhi","CrePhi_adj_output.json", "CrePhi_Reply_adj_output.json","CrePhi_GPT2Emails.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
GPT2EmailsTable = TSVform(SocEngGPT2Total,
                          MalwareGPT2Total,
                          ReconnGPT2Total,
                          CrePhiGPT2Total,
                          "Social_Engineering-800_result.json",
                          "Malware-800_result.json",
                          "Reconnaissance-800_result.json",
                          "Credential_Phishing-800_result.json",
                          "GPT2Emails.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
GPT2EmailsTable

,AttackType_index,EmailText,SenderFace,PhishIrisPath,Caption
0,SocEng. 001-1,From: Jean Charles\nTitle: Urgent M/F\n Urgent...,Face_generator-1600 result/face_generator-0001...,Social_Engineering-800_result/social_engineeri...,a large body of water next to a body of water;...
1,SocEng. 002-1,From: Mr.Malick Camara\nTitle: Business Invita...,Face_generator-1600 result/face_generator-0002...,Social_Engineering-800_result/social_engineeri...,a person holding a cell phone in their hands; ...
2,SocEng. 003-1,From: MARTIN CHIME\nTitle: URGENT BUSINESS MA...,Face_generator-1600 result/face_generator-0003...,Social_Engineering-800_result/social_engineeri...,a bird is flying over a body of water; a large...
3,SocEng. 004-1,From: Mrs Farah Al-hashemi\nTitle: Attention B...,Face_generator-1600 result/face_generator-0004...,Social_Engineering-800_result/social_engineeri...,a large white bird standing on top of a beach;...
4,SocEng. 005-1,From: Toe Brown\nTitle: URGENT\n URGENT ATTN ...,Face_generator-1600 result/face_generator-0005...,Social_Engineering-800_result/social_engineeri...,a large white and blue surfboard sitting on to...
...,...,...,...,...,...
2395,CrePhi. 196-3,From: lucman_yengeni2@yahoo.co.uk\nTitle: Re:...,Face_generator-1600 result/face_generator-0796...,NaN,NaN
2396,CrePhi. 197-3,From: alhaji assir\nTitle: Re: Re: URGENT BUSI...,Face_generator-1600 result/face_generator-0797...,NaN,NaN
2397,CrePhi. 198-3,From: michael.bolema@mail2world.com\nTitle: Re...,Face_generator-1600 result/face_generator-0798...,NaN,NaN
2398,CrePhi. 199-3,From: TD Amery\nTitle: Re: Re: Requirements\n\...,Face_generator-1600 result/face_generator-0799...,NaN,NaN


### 9.
### Generating Grover training input and testing input

files needed  (you should put those files into the same folder with this notebook)
 - gpt2_gentext_20210325_230034_socEng.txt  
 - gpt2_gentext_20210325_234254_malware.txt  
 - gpt2_gentext_20210330_063140_reconn.txt  
 - gpt2_gentext_20210330_052026_cre_phi.txt  
 - Grover_Fradulent.json  
 - GPT2Emails.tsv  

files created
 - GroverTrainingInput.jsonl(use in Grover)  
 - GroverTestingInput_initial.jsonl(use in Grover)  
 - GroverTestingInput_all.jsonl(use in Grover)  


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import json
from google.colab import files 
import pandas as pd

In [ ]:
## generating training input for Grover. total 3992 emails from fradulent email dataset (human made) and 1200 emails from GPT2 model.
def trainingInput(SocEng,Malware,Reconn,CrePhi,Fradulent,ResultFile):

    with open(SocEng,"r",encoding='utf8') as f:
        SocEngTrainingList = f.read().split("\n====================\n")
    with open(Malware,"r",encoding='utf8') as f:
        MalwareTrainingList = f.read().split("\n====================\n")
    with open(Reconn,"r",encoding='utf8') as f:
        ReconnTrainingList = f.read().split("\n====================\n")
    with open(CrePhi,"r",encoding='utf8') as f:
        CrePhiTrainingList = f.read().split("\n====================\n")
    with open(Fradulent) as f:
        Fradulent = json.load(f)

    MachineTrainingList = []
    GPT2result = SocEngTrainingList[0:300] + MalwareTrainingList[0:300] + ReconnTrainingList[0:300] + CrePhiTrainingList[0:300]
    for i in range(len(GPT2result)):
        MTDict = {}
        MTDict["article"] = GPT2result[i]
        MTDict["label"] = "machine" ## human or machine
        MTDict["split"] = "train"
        MachineTrainingList.append(MTDict)  
    GroverJSONL = Fradulent + MachineTrainingList

    with open(ResultFile, 'w') as f:
        for email in GroverJSONL:
            json.dump(email, f)
            f.write('\n')
    files.download(ResultFile)

## generating testing input for Grover.
def TestingInput(GPT2EmailsTSV, SaveAsJSONL, InitialEmailOnly = True):
    GPT2EmailsTable = pd.read_csv(GPT2EmailsTSV,sep="\t",index_col=[0])
    TestingList = []
    for i in range(len(GPT2EmailsTable)):
        TestingDict = {}
        if InitialEmailOnly == True and (GPT2EmailsTable.iloc[i,0].find("-2") >= 0  or GPT2EmailsTable.iloc[i,0].find("-3") >= 0 ):
            continue
        else:
            content = readFromBody(GPT2EmailsTable.iloc[i,1])
            TestingDict["article"] = content
            TestingDict["label"] = "machine" ## human or machine
            TestingDict["split"] = "test"       
            TestingList.append(TestingDict) 
    with open(SaveAsJSONL, 'w') as f:
        for email in TestingList:
            json.dump(email, f)
            f.write('\n')
    files.download(SaveAsJSONL)
    return(TestingList)

## read text from the body of the email 
def readFromBody(content):
    content2 = content[content.find("\n")+1:]
    content3 = content2[content2.find("\n")+1:]
    return(content3)


In [ ]:
## generating Grover training input (GroverInput.jsonl)
trainingInput("gpt2_gentext_20210325_230034_socEng.txt",
              "gpt2_gentext_20210325_234254_malware.txt",
              "gpt2_gentext_20210330_063140_reconn.txt",
              "gpt2_gentext_20210330_052026_cre_phi.txt",
              "Grover_Fradulent.json",
              "GroverTrainingInput.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
GroverTestingInput_initial = TestingInput("GPT2Emails.tsv","GroverTestingInput_initial.jsonl", InitialEmailOnly = True)
GroverTestingInput_all = TestingInput("GPT2Emails.tsv","GroverTestingInput_all.jsonl", InitialEmailOnly = False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 10.
### merge Grover result into GPT2Emails and
### Add columns and rows in our final TSV file

* Need to run **Grover** outside prior to this step  
(**Grover:** TEAM_MIMECRAFT_DSCI550_HW_EXTRACT\new_attacks\Grover/Grover_training.ipynb)  
after running Grover_training.ipynb, bring the final input(2400flags.txt) on this folder.
* Instead of running Grover-training.ipynb, use result file from our github repository (https://github.com/shlee55/550Repository.git).   
Path of the file: 550Repository/get_flags/get_flags/2400flags.txt

files needed  (you should put those files into the same folder with this notebook)
 - featureTable_temp.tsv  
 - GPT2Emails.tsv
 - 2400flags.txt (github-result from Grover)

files created
 - new_attacks.tsv
 - featureTable_v2.tsv(final result)

In [ ]:
!cd /content && git clone 'https://github.com/shlee55/550Repository.git'

fatal: destination path '550Repository' already exists and is not an empty directory.


In [ ]:
### every step will be starting with import part in case of running from the specific step. 
import os
import pandas as pd
from google.colab import files

In [ ]:
## load Grover output from repository
def Grover(GroverTXT):
    GroverList = []
    with open (GroverTXT)as f:
        for line in f:
            GroverList.append(line.strip())
    GroverTable = pd.DataFrame({'col':GroverList})
    GroverTable.columns = ["Grover"]
    return(GroverTable)

## combine every features into final tsv file
def editTSV(featureTableTSV, GPT2EmailsTSV,featureTSV,newAttackTSV,GroverTXT):
    FeatureTable = pd.read_csv(featureTableTSV,sep="\t",index_col=[0])
    GPT2Emails = pd.read_csv(GPT2EmailsTSV,sep="\t",index_col=[0])
    GroverTable = Grover(GroverTXT)
    GPT2Emails = pd.concat([GPT2Emails,GroverTable], axis=1)
    GPT2Emails = GPT2Emails.sort_values(by=['AttackType_index'],ignore_index=True)
    GPT2Emails.to_csv(newAttackTSV, sep = '\t')
    files.download(newAttackTSV)
    FinalTSV = pd.concat([FeatureTable,GPT2Emails], axis=0,ignore_index=True)
    FinalTSV.to_csv(featureTSV, sep = '\t')
    files.download(featureTSV)
    return(FinalTSV,GPT2Emails)


In [ ]:
FinalTSV, new_attack = editTSV("featureTable_temp.tsv",
                   "GPT2Emails.tsv",
                   "featureTable_v2.tsv",
                   "new_attacks.tsv",
                   "2400flags.txt"
                   )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_attack.head()

,AttackType_index,EmailText,SenderFace,PhishIrisPath,Caption,Grover
0,CrePhi. 001-1,From: jimmy gill\nTitle: urg.at\n urg.at ATTN:...,Face_generator-1600 result/face_generator-0601...,Credential_Phishing-800_result/credential_phis...,a boat is floating in the water near a shore; ...,Machine
1,CrePhi. 001-2,"Title: Re: urg.at\nDear jimmy gill,\n\nPlease ...",Face_generator-1600 result/face_generator-1401...,NaN,NaN,Machine
2,CrePhi. 001-3,From: jimmy gill\nTitle: Re: Re: urg.at\n\n> \...,Face_generator-1600 result/face_generator-0601...,NaN,NaN,Machine
3,CrePhi. 002-1,From: Mr. Richard Lester.\nTitle: GOOD DAY......,Face_generator-1600 result/face_generator-0602...,Credential_Phishing-800_result/credential_phis...,a group of people riding on top of a wave; a g...,Machine
4,CrePhi. 002-2,Title: Re: GOOD DAY.................\nDear Mr....,Face_generator-1600 result/face_generator-1402...,NaN,NaN,Machine


In [ ]:
##sample 
FinalTSV[3988:4000]

,filename,AttackType:Reconnaissance,AttackType:SocialEngineering,AttackType:Malware,AttackType:CredentialPhishing,Attacker Title,Urgency of the attack email,DateAndTime:Date,DateAndTime:time,Attacker Offering,AttackerLocation:Country,AttackerLocation:Region,AttackerLocation:City,AttackerLocation:Country From Content,Attacker Relationship,AttackerSentiment:polarity,AttackerSentiment:subjectivity,AttackerLanguageStyle:misspelling,AttackerLanguageStyle:randomCapital,Estimated Age,Attacker IP Known As Phisher,Total Deaths and Injuries,Total Damage,Disaster Count,GDP per capita,Internet Use,Age dependency,URL Label,URL Length,URL Classification,XHTML,EmailText,AttackType_index,SenderFace,PhishIrisPath,Caption,Grover
3988,996.json,False,False,False,False,no title exist,2.0,2004-07-10,01:10,Money,South Africa,Unknown,Unknown,South Africa,stranger,0.10,0.38,2.0,2.0,33.0,unknown,464473.0,48757.0,89.0,6472.11,8.43,56.27,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: Mr. Dicka Dumas\nTitle: FROM:Mr.D Dumas....,NaN,NaN,NaN,NaN,NaN
3989,997.json,False,False,False,False,no title exist,2.0,2004-07-10,01:58,Money,South Africa,Unknown,Unknown,South Africa,stranger,0.10,0.38,2.0,2.0,32.0,unknown,464473.0,48757.0,89.0,6472.11,8.43,56.27,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: Mr. Dicka Dumas\nTitle: FROM:Mr.D Dumas....,NaN,NaN,NaN,NaN,NaN
3990,998.json,False,False,False,True,native,2.0,2004-07-07,09:41,Money,South Africa,Unknown,Unknown,South Africa,stranger,0.08,0.47,1.0,2.0,33.0,unknown,464473.0,48757.0,89.0,6472.11,8.43,56.27,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: DR. EKIYOH MARTINS\nTitle: BUSINESS PROP...,NaN,NaN,NaN,NaN,NaN
3991,999.json,True,False,False,False,merchant,2.0,2004-07-09,17:23,Money,Malaysia,Unknown,Unknown,Malaysia,friend of friend,0.07,0.46,1.0,1.0,29.0,unknown,464473.0,48757.0,89.0,7720.83,42.25,54.81,NaN,NaN,NaN,"<html xmlns=""http://www.w3.org/1999/xhtml"">\n<...",From: Mr.Harry Hakeem\nTitle: CHARITY SUPPORT\...,NaN,NaN,NaN,NaN,NaN
3992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From: jimmy gill\nTitle: urg.at\n urg.at ATTN:...,CrePhi. 001-1,Face_generator-1600 result/face_generator-0601...,Credential_Phishing-800_result/credential_phis...,a boat is floating in the water near a shore; ...,Machine
3993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Title: Re: urg.at\nDear jimmy gill,\n\nPlease ...",CrePhi. 001-2,Face_generator-1600 result/face_generator-1401...,NaN,NaN,Machine
3994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From: jimmy gill\nTitle: Re: Re: urg.at\n\n> \...,CrePhi. 001-3,Face_generator-1600 result/face_generator-0601...,NaN,NaN,Machine
3995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From: Mr. Richard Lester.\nTitle: GOOD DAY......,CrePhi. 002-1,Face_generator-1600 result/face_generator-0602...,Credential_Phishing-800_result/credential_phis...,a group of people riding on top of a wave; a g...,Machine
3996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Title: Re: GOOD DAY.................\nDear Mr....,CrePhi. 002-2,Face_generator-1600 result/face_generator-1402...,NaN,NaN,Machine
3997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From: Mr. Richard Lester.\nTitle: Re: Re: GOO...,CrePhi. 002-3,Face_generator-1600 result/face_generator-0602...,NaN,NaN,Machine
